In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime, date
import plotly as py
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.offline as offline
import os
plt.style.use('fivethirtyeight') # For plots

df_holiday = pd.read_csv('calendar.csv')
df_holiday['ds'] = pd.to_datetime(df_holiday["ds"]).dt.strftime('%Y-%m-%d')
df_holiday[['ds', 'holiday']] = df_holiday[['ds', 'holiday']]
df_holiday = df_holiday[['ds', 'holiday']]

/Users/dizzz/opt/anaconda3/envs/new/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



In [24]:
packs = pd.read_csv('sales.csv')

packs['ds'] = pd.to_datetime(packs['ds'])
model = Prophet(
    seasonality_mode="multiplicative",
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=80,
    changepoint_prior_scale = 0.097,
    interval_width = 0.97,
    holidays = df_holiday
    )
model.add_country_holidays(country_name='RU')
model.fit(packs)
future = model.make_future_dataframe(periods = 30, freq = 'd')
forecast = model.predict(packs)
forecast1 = model.predict(future)
forecast_packs = forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


Initial log joint probability = -23.9809
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       864.715     0.0155753       270.646       2.417      0.2417      126   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       896.944    0.00771897       115.867           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       903.311    0.00787413       402.399      0.6926      0.6926      364   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       905.481   0.000860046       266.439      0.4373      0.4373      485   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       906.906    0.00096313       73.6208      0.5793      0.5793      603   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [6]:
test = forecast_packs.merge(packs, on=["ds"])
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100
actual = test['y']
pred = test['yhat']
mape(actual, pred)

16.351764049812694

In [8]:
#print performance metrics
print("MAE yhat\t: {}\nMAE yhat\t: {}\nMAE yhat_lower: {}\nMAE yhat_upper: {}".format(
    mean_absolute_error(test['y'].values,test['yhat']),
    mean_absolute_error(test['y'].values,test['yhat']),
    mean_absolute_error(test['y'].values,test['yhat_lower']),
    mean_absolute_error(test['y'].values,test['yhat_upper'])))

MAE yhat	: 84.74534649794973
MAE yhat	: 84.74534649794973
MAE yhat_lower: 308.8788516688831
MAE yhat_upper: 306.8909310997045
